# Task E
_Use the location data collected in step A to find out lifts and sentiments regarding the candidates in large versus small cities/towns in Texas._

In [1]:
import numpy as np
import pandas as pd

Read in all tweets into dataframs.

In [2]:
tweets = pd.read_csv('tot_tweets.csv')

In [3]:
tweets = tweets.drop(columns=['id'])

In [4]:
tweets.shape

(5741, 2)

In [5]:
tweets.head()

,location,text
0,"Tennessee, USA","b""RT @AliAdair22: \xf0\x9f\x90\xa6Next, Beto O..."
1,NaN,"b""RT @AliAdair22: \xf0\x9f\x90\xa6Next, Beto O..."
2,NaN,"b""Ted Cruz, Beto O'Rourke try to rally Latino ..."
3,America,b'RT @RonNehring: Third poll now showing Cruz ...
4,West Texas,"b'Beto O\xe2\x80\x99Rourke, the Democratic con..."


#### Name replacement

In [6]:
# Let's write a function to take care of the names
def replace_names(text):
    
    text = text.lower()
    
    beto_words = ['@betoorourke' , "beto o'rourke", "o'rourke", 'rourke']
    cruz_words = ['@tedcruz', 'ted cruz', 'ted']
    
    for w in beto_words:
        try:
            text = text.replace(w, 'beto')
        except:
            pass
    for w in cruz_words:
        try:
            text = text.replace(w, 'cruz')
        except:
            pass
    
    return text

In [7]:
# apply replace_names() on each tweet
text_column = []
for t in tweets.itertuples():
    text = t.text
    new_text = replace_names(text)
    text_column.append(new_text)

In [8]:
tweets.text = text_column

#### City stuff

In [9]:
# Let's get a list of what we'll consider big cities/towns in Texas.
large_cities = open('large_cities.txt', 'r')
cities = []
for line in large_cities:
    l = line.strip()
    l_list = l.split()
    if l_list[2].isalpha():
        city = l_list[1] + ' ' + l_list[2]
    else:
        city = l_list[1]
    cities.append(city)
cities = cities[0:11]
print (cities)

['Houston', 'San Antonio', 'Dallas', 'Austin', 'Fort Worth', 'El Paso', 'Arlington', 'Corpus Christi', 'Plano', 'Laredo', 'Lubbock']


In [10]:
# get column for location that is either Big, Small, or none
location_column = []
for t in tweets.itertuples():
    if type(t.location) == str:
        for c in cities:
            if c in t.location:
                location_column.append('Big')
                break
            elif c == cities[-1] and ', TX' in t.location:
                location_column.append('Small')
                break
            elif c == cities[-1]:
                location_column.append('none')
                break
    else:
        location_column.append('none')

In [11]:
# replace old location column with new
tweets.location = location_column

In [12]:
tweets.shape

(5741, 2)

In [13]:
tweets.head(6)

,location,text
0,none,"b""rt @aliadair22: \xf0\x9f\x90\xa6next, beto b..."
1,none,"b""rt @aliadair22: \xf0\x9f\x90\xa6next, beto b..."
2,none,"b""cruz, beto try to rally latino voters in tex..."
3,none,b'rt @ronnehring: third poll now showing cruz ...
4,none,"b'beto o\xe2\x80\x99beto, the democratic congr..."
5,Big,b'rt @wfaa: does texas\xe2\x80\x99 senate race...


#### Calculate Lift for Candidate / City size

Get count of tweets mentioning Beto / Cruz, and count of all tweets with Beto and/or Cruz in it

In [14]:
beto_count = 0
cruz_count = 0
n_tweets = 0
for t in tweets.itertuples():
    if 'beto' in t.text and 'cruz' in t.text:
        beto_count += 1
        cruz_count += 1
        n_tweets += 1
    elif 'beto' in t.text:
        beto_count += 1
        n_tweets += 1
    elif 'cruz' in t.text:
        cruz_count += 1        
        n_tweets += 1

Get count of tweets from large / small cities in Texas

In [15]:
big_city_count = 0
small_city_count = 0
for t in tweets.itertuples():
    if t.location == 'Big':
        big_city_count += 1
    elif t.location == 'Small':
        small_city_count += 1

Get count of tweets with Beto/Big, Beto/Small, Cruz/Big, Cruz/Big

In [16]:
bb_count = 0
bs_count = 0
cb_count = 0
cs_count = 0
for t in tweets.itertuples():
    if 'beto' in t.text and t.location == 'Big':
        bb_count += 1
    if 'beto' in t.text and t.location == 'Small':
        bs_count += 1
    if 'cruz' in t.text and t.location == 'Big':
        cb_count += 1
    if 'cruz' in t.text and t.location == 'Small':
        cs_count += 1

Calculate lift Beto/Big, Beto/Small, Cruz/Big, Cruz/Big

In [18]:
candidate_counts = [beto_count, cruz_count]
city_counts = [big_city_count, small_city_count]
combo_counts = [bb_count, bs_count, cb_count, cs_count]
lifts = [] # order: [0] beto vs big, [1] beto vs small, [2] cruz vs big, [3] cruz vs small
i = 0
for cand_count in candidate_counts:
    for city_count in city_counts:
        lifts.append( (len(tweets) * combo_counts[i]) / (cand_count * city_count) )
    i+=1

In [24]:
lifts = np.reshape(lifts, (2, 2)).T # reshape for similarities matrix

In [28]:
similarities = pd.DataFrame(lifts, columns=['beto', 'cruz'], index=['big city', 'small city'])

In [29]:
similarities

,beto,cruz
big city,1.013855,0.354937
small city,2.945009,1.031009
